# Measure Tracking Error for each drive axis (and with no motion!)

In [ ]:
import sys
import asyncio
import time
import os

import numpy as np
import logging 
import yaml
import matplotlib.pyplot as plt
import astropy

from lsst.ts import salobj
from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign
from lsst.ts.observatory.control.utils import RotType

from lsst.ts.idl.enums.Script import ScriptState

from lsst.ts.observing.utilities.decorated_logger import DecoratedLogger

In [ ]:
logger = DecoratedLogger.get_decorated_logger()
logger.level = logging.DEBUG

## Instantiate the focus Script

First derive and index for the script that is specific to your user

In [ ]:
logger.info(f'Your UID is {os.getuid()}')
index=os.getuid()*10+np.random.randint(0,9)
logger.info(f'The generated index is {index}')

Instantiate the script then start all remotes

In [ ]:
script = LatissCWFSAlign(index=index, remotes=True)  # this essentially calls the init method
await script.start_task

Set the script to have a DEBUG log level

In [ ]:
script.log.level = logging.DEBUG

### Slew to a target to focus the telescope
This is not part of the actual script but seems to come in handy, so the code is here in the notebook for convenience

Print the available instrument filter and gratings. <br>
This is useful both for slewing and for the configuration steps below

In [ ]:
inst_setup = await script.latiss.get_available_instrument_setup()
logger.info(f'filters are: {inst_setup[0]},\ngratings are: {inst_setup[1]}')

## Target Selection 
For Azimuthally dominated motion the target should be transiting the meridian. <br>
For elevation dominated motion the target will be in the southeast. <br>
For no motion, point to the south celestial pole (TYC 9502-41-1 (RA = 06 02 13.69, dec = -86 17 28, Vmag = 12)

#### Find/Specify a target based on the Az/El and slew to it

In [ ]:
# target = await atcs.find_target(160,60,mag_limit=6.0)

In [ ]:
# HD 213792 = HIP 111416 (V=8.5 F7V, declination = -22 36 07) rising towards the meridian. 
#If saturated then do an offset or else slew to TYC 6426-1727-1, Vmag=11., fainter star. 
target = "HD 213792"

In [ ]:
await script.atcs.slew_object(target, rot_type=RotType.PhysicalSky)

Check that the target is in the field and is below ~20000 ADU to avoid saturation in a 30s image

In [ ]:
await script.latiss.take_engtest(10)

Offset the telescope (if required). <br>
Offsets are in detector X/Y coordinates and in arcseconds

In [ ]:
# await script.atcs.offset_xy(x=20, y=20)

## Configure the script

In [ ]:
configuration = yaml.safe_dump({"filter": 'FELH0600', 
                                "grating": 'empty_1',
                                "exposure_time": 20,})

Set script state to UNCONFIGURED. <br>
The next line is not required the first time the script is run, however, in each additional instance the cell is run, an error will be thrown if it is not included. <br>
Therefore, it is included here despite being a non-operation in the first instance.

In [ ]:
script.set_state(ScriptState.UNCONFIGURED)

#### Put the ScriptState to CONFIGURED

In [ ]:
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

Set these script parameters to None to verify a re-reduction does not happen of the images

In [ ]:
script.intra_visit_id = None
script.extra_visit_id = None
script.short_timeout = 10

### Set groupID and launch the script
This sets the same group ID for all exposures taken in the script

In [ ]:
group_id_data = script.cmd_setGroupId.DataType(
                groupId=astropy.time.Time.now().isot
            )
await script.do_setGroupId(group_id_data)
await script.arun()

## Telescope is now collimated and in focus

#### Grab focus offsets incase something goes awry and we need to come back into focus

In [ ]:
original_focus_offset = await atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(original_focus_offset)

## Take a stack of n x 30 sec images in SDSSg

In [ ]:
science_image = await latiss.take_engtest(exptime=30, n=10, filter='SDSSg',grating='empty_1')
print ('SDSSg stack Sequence_image = ' + str(science_image))